## ISO Country Code Test

Overview: The International Organization for Standardization maintains a list of country codes. This script will initiate a public API request to verify the country codes under test. 

Task:

Below outlines the steps to be performed:

1) Import the necessary libraries for the project.
2) Define the functions that will faciliate the vies check.
3) Send request to API and test for succesful response. 
4) Parse the api response content into a pandas dataframe.
5) Verify script with test data.

In [7]:
import requests
import json
import pandas as pd
import re

API_URL = 'https://restcountries.com/v3.1/all'
COLUMNS_RESP = ["common","official", "cca2", "cca3","continents"]

def _api_request():
    # Send API request.
    response_API = requests.get(API_URL)
    json_file = json.loads(response_API.text)
    data = pd.DataFrame.from_dict(json_file)
    
    # Transform response and store in lookup table.
    data['common'] = [x['common'] for x in data['name']]
    data['official'] = [x['official'] for x in data['name']]
    df = pd.DataFrame(data,columns=COLUMNS_RESP)
    df['continents'] = [x[0] for x in df['continents']]
    df['cca2'] = df['cca2'].replace('GB','XU')
    return df

def country_to_iso2(df, column_name):
    # Match country name to ISO-2 equivalent. Append result. 
    df_out = _api_request()[['common','cca2']]
    df_out = pd.merge(df, df_out, left_on=column_name, right_on='common', how='left')
    df_out.drop('common', axis=1, inplace=True)
    df_out = df_out.rename(columns={'cca2': 'iso2'})
    return df_out

def alpha3_to_iso2(df, column_name):
    # Match alpha-3 country code to ISO-2 equivalent. Append result.
    df_out = _api_request()[['cca3','cca2']]
    df_out = pd.merge(df, df_out, left_on=column_name, right_on='cca3', how='left')
    df_out.drop('cca3', axis=1, inplace=True)
    df_out = df_out.rename(columns={'cca2': 'iso2'})
    return df_out

def alpha2_to_iso2(df, column_name):
    # Match alpha2 country code to ISO-2 equivalent. Append result.
    df[column_name] = df[column_name].replace(regex="UK|GB", value='XU')
    df_out = _api_request()['cca2']
    df_out = pd.merge(df, df_out, left_on=column_name, right_on='cca2', how='left')
    df_out = df_out.rename(columns={'cca2': 'iso2'})
    return df_out

def main(): 
    data1 = ['Spain','France','Italy','United Kingdom','Slovenia']
    data2 = ['ESP','FRA','ITA','GBR','SVN']
    data3 = ['ES','FR','IT','UK','SI']
    df = pd.DataFrame(data1,columns=['Country'])
    display(country_to_iso2(df,'Country'))
        
if __name__ == '__main__':
    main()

,Country,iso2
0,Spain,ES
1,France,FR
2,Italy,IT
3,United Kingdom,XU
4,Slovenia,SI
